In [1]:
import gym
import tensorflow as tf
import numpy as np

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [3]:
env = gym.make('BipedalWalker-v2')

[2017-09-10 12:30:40,179] Making new env: BipedalWalker-v2


In [15]:
# n_actions = env.action_space.n
# n_states = env.observation_space.shape
print(env.action_space.sample)
print(env.observation_space.sample)
print(env.observation_space.high)
print(env.observation_space.low)

<bound method Box.sample of Box(4,)>
<bound method Box.sample of Box(24,)>
[ inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf  inf
  inf  inf  inf  inf  inf  inf  inf  inf  inf]
[-inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf -inf
 -inf -inf -inf -inf -inf -inf -inf -inf -inf]


In [11]:
s = env.reset()
for i in range(10000):
    new_s, reward, done, _ = env.step(env.action_space.sample())
    env.render()
    if done:
        break
#print(i)
env.close()

In [29]:
agent = Sequential()
agent.add(Dense(20, input_dim=24, activation='relu'))
agent.add(Dense(20, activation='relu'))
agent.add(Dense(4, activation='tanh'))
agent.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

In [30]:
s = env.reset()
print(agent.predict(s.reshape(1, 24))[0])

[-0.47961742  0.33022529 -0.52494264  0.14597335]


In [31]:
t_max = 1000
def generate_sample():
    s = env.reset()
    batch_s = []
    batch_a = []
    total_reward = 0
    
    for i in range(t_max):
        a = agent.predict(s.reshape(1, 24))[0]
        new_s, r, done, _ = env.step(a)
        batch_s.append(s)
        batch_a.append(a)
        s = new_s
        total_reward = total_reward + r
        if done:
            break
    env.close()
    return batch_s, batch_a, total_reward

In [32]:
iterations = 100
percentile = 70
samples = 250

for i in range(iterations):
    population = [generate_sample() for i in range(samples)]
    batch_states,batch_actions,batch_rewards = map(np.array,zip(*population))
    threshold = np.percentile(batch_rewards, percentile)
    elite_states = batch_states[batch_rewards > threshold]
    elite_actions = batch_actions[batch_rewards > threshold]
    elite_states, elite_actions = map(np.concatenate, [elite_states, elite_actions])
    agent.fit(epochs=1, x=elite_states, y=elite_actions)
    print('Iteration: {0}, Mean Reward: {1:.2f}, Threshold: {2:.2f}'.format(i + 1, np.mean(batch_rewards), threshold))

KeyboardInterrupt: 

In [28]:
s = env.reset()
t_max = 1000
for i in range(t_max):
    env.render()
    a = agent.predict(s.reshape(1, 24))[0]
    new_s, r, done, _ = env.step(a)
    s = new_s
    if done:
        break
env.close()